<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [3]:
import pandas as pd
import numpy as np

In [4]:
data = {'x1': [0,0,1,0,1,1,0],
        'x2': [0,1,0,1,0,1,0],
        'x3': [1,1,1,0,0,1,0],
        'y':  [0,1,1,1,1,0,0]
       }
df = pd.DataFrame.from_dict(data).astype('int')

In [5]:
X = df[['x1', 'x2', 'x3']].values
y = df[['y']].values

In [6]:
class NeuralNetwork:
    def __init__(self):
        # Architecture of my NN
        self.inputs = 3
        self.hidden = 4
        self.output = 1

        # Initial Weights
        # 3x4 Matrix Array for the First Layer
        self.weights1 = np.random.rand(self.inputs, self.hidden)
       
        # 4x1 Matrix Array for Hidden to Output
        self.weights2 = np.random.rand(self.hidden, self.output)
        
    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))
    
    def sigmoid_derivative(self, s):
        return s * (1 - s)
    
    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward.
        """
        
        # Weighted sum of inputs => hidden layer
        
        self.first_weighted_sum = np.dot(X, self.weights1)
        
        # Activations of weighted sum
        self.activated_hidden = self.sigmoid(self.first_weighted_sum)
        
        self.second_weighted_sum = np.dot(self.activated_hidden, self.weights2)
        
        # Final activation of output
        self.activated_output = self.sigmoid(self.second_weighted_sum)
        
        return self.activated_output
    
    def backpropagate(self, X, y):
        """
        Backward propagate through the network
        """
        
        # Error in Output
        self.error_output = y - self.activated_output
        
        # Apply Derivative of Sigmoid to error
        # How far off are we in relation to the Sigmoid f(x) of the output
        # ^- aka hidden => output
        self.adjust_output = self.error_output * self.sigmoid_derivative(self.activated_output)
        
        # z2 error
        self.hidden_error = self.adjust_output.dot(self.weights2.T)
        
        # Descent Portion
        # How much of that "far off" can explained by the input => hidden
        self.adjust_hidden = self.hidden_error * self.sigmoid_derivative(self.activated_hidden)
        
        # Adjustment to first set of weights (input => hidden)
        self.weights1 += X.T.dot(self.adjust_hidden)
        # Adjustment to second set of weights (hidden => output)
        self.weights2 += self.activated_hidden.T.dot(self.adjust_output)   
    
    def train(self, X, y, iter = 10):
        for epoch in range(iter):
            self.feed_forward(X)
            self.backpropagate(X,y)
        return self
        # return self.__dict__
    
    def predict(self, X):
        return np.round(self.feed_forward(X))
#         return [1 if prediction>0.5 else 0 for prediction in self.feed_forward(X)]

In [7]:
nn = NeuralNetwork()
nn.train(X,y, iter=1000)

In [61]:
nn.predict(X)

array([[0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.]])

In [63]:
from sklearn.metrics import accuracy_score

accuracy_score(nn.predict(X), y)

1.0

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


### Parts
1. Gathering & Transforming the Data
2. Making MNIST a Binary Problem
3. Estimating your Neural Network (the part you focus on)

### Gathering the Data 

`keras` has a handy method to pull the mnist dataset for you. You'll notice that each observation is a 28x28 arrary which represents an image. Although most Neural Network frameworks can handle higher dimensional data, that is more overhead than necessary for us. We need to flatten the image to one long row which will be 784 values (28X28). Basically, you will be appending each row to one another to make on really long row. 

In [39]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [2]:
# input image dimensions
img_rows, img_cols = 28, 28

In [31]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [32]:
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

# Normalize Our Data
x_train = x_train / 255
x_test = x_test / 255

In [16]:
# Now the data should be in a format you're more familiar with
x_train.shape

(60000, 784)

### Making MNIST a Binary Problem 
MNIST is multiclass classification problem; however we haven't covered all the necessary techniques to handle this yet. You would need to one-hot encode the target, use a different loss metric, and use softmax activations for the last layer. This is all stuff we'll cover later this week, but let us simply the problem for now: Zero or all else.

In [33]:
import numpy as np

y_temp = np.zeros(y_train.shape)
y_temp[np.where(y_train == 0.0)[0]] = 1
y_train = y_temp

y_temp = np.zeros(y_test.shape)
y_temp[np.where(y_test == 0.0)[0]] = 1
y_test = y_temp

In [40]:
# A Nice Binary target for ya to work with
y_train

array([0., 1., 0., ..., 0., 0., 0.])

### Estimating Your `net

In [ ]:
##### Your Code Here #####



## Stretch Goals: 

- Make MNIST a multiclass problem using cross entropy & soft-max
- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?